### Räumlicher Plot legaler Graffittiflächen in Dresden

Datensatz auf opendata.dresden.de: https://opendata.dresden.de/informationsportal/?open=1&result=3B7136B1ADF14E59A2E74D3FACE95C2C#app/mainpage/

In [ ]:
# imports
import geopandas
import shapely

In [ ]:
graffitti_areas = geopandas.read_file('../data/legale_graffitiflaechen.csv')
graffitti_areas

In einem GeoPandas `GeoDataFrame` repräsentiert jede Zeile ein räumliches Objekt (-> Feature). Jedes Objekt hat eine ID, eine beliebige Anzahl weiterer Eigenschaften (Attribute) sowie eine räumliche Definition (automatisch angelegte Spalte `geometry`), siehe https://geopandas.org/en/stable/getting_started/introduction.html.

<img src="../assets/geodataframe.png" width=450px>



Die eingeladene CSV besitzt bereits eine Spalte mit Objekt IDs (`id`), welche jedoch nicht automatisch als solche erkannt wird (daran zu sehen, dass eine neue Spalte ganz links angelegt wurde). Eine Spalte kann mittels `set_index()` als ID-Spalte definiert werden. 

Die `geometry`-Spalte ist mit `None` gefüllt. Die Werte müssen manuell aus der in der CSV vorhandenen Spalte `geom` ausgelesen werden. In `geom` sind zwei Informationen festgehalten, 1) die ID des jeweils verwendeten Koordinatenreferenzsystems (SRID) und 2) die Koordinaten. Die beiden Informationen sind mit Semikolon getrennt und SRID ist für jede Zeile im Datensatz gleich. Die Koordinaten sind im WKT-Format (Well-know Text, https://de.wikipedia.org/wiki/Simple_Feature_Access) angegeben.

Schritte zur Definition der `geometry`-Spalte:
1) Koordinatenreferenzsystem angeben
    - Methode `set_crs()` -- nimmt eine Koordinatenreferenzsystem-ID entgegen
    - Koordinatenreferenzsystem-ID aus erster Spalte extrahieren (`graffitti_areas_gdf['geom'][0].split(';')[0].split('=')[1]`)
2) Feature-Geometries definieren
    - die Spalte `geometry` muss Objekte der Klasse `shaleply.Geometry` (https://shapely.readthedocs.io/en/stable/geometry.html) enthalten
    - die Python Bibliothek `shapely` hat die Methode `from_wkt()` , welche ein `shapely.Geometry` Objekt aus einem WKT-String erzeugt
    - wir iterieren also durch die exisiterende `geom`-Spalte, extrahieren die WKT-Strings, wandeln diese in eine `shapely.Geometery` um, und schreiben diese dann in eine neue Liste, welche wir final als `geometry`-Spalte definieren. 

In [ ]:
# set index column
graffitti_areas = graffitti_areas.set_index('id')
# extract CRS code from first column in `geom`
graffitti_areas = graffitti_areas.set_crs(graffitti_areas['geom'][0].split(';')[0].split('=')[1])
# create list that only contains Shapely Geometries from WKT-Strings in `geom` column
graffitti_areas.geometry = [shapely.from_wkt(geom.split(';')[1]) for geom in graffitti_areas['geom']]
# # remove geom column
graffitti_areas = graffitti_areas.drop(columns='geom')
graffitti_areas

In [ ]:
# explore()-documentation: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
graffitti_areas.explore(
    tooltip=["adresse", "bemerkung", "groesse"],
    tooltip_kwds=dict(labels=False),
    color="tomato", 
    marker_kwds=dict(radius=10, fill=True),
    width =800,
    height = 500,
    tiles="Stamen Toner"
)